In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/aula_09"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    !python3.8 -m pip install openai -q
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_09/"    

In [4]:
import pandas as pd
import pickle
import numpy as np

import json

import time

import re

import openai

In [5]:
TREC_COVID_MERGED_FILE="trec_covid_merged_data.tsv"
TREC_COVID_DOCUMENTS_FILE="trec_covid_original_title_text_merged.tsv"

TREC_COVID_QRELS="trec_covid_qrels.tsv"

In [6]:
API_KEYS_FILE="../api_keys_20230324.json"

In [7]:
pd.set_option('display.max_colwidth', None)

## Set the random seed

In [8]:
RANDOM_SEED = 6

In [9]:
rng = np.random.default_rng(RANDOM_SEED)

## Explore TREC COVID documents with questions

In [10]:
merged_df = pd.read_csv(TREC_COVID_MERGED_FILE, sep="\t")

In [11]:
merged_df

,query-id,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
0,1,005b2j4b,2,what is the origin of COVID-19,Monophyletic Relationship between Severe Acute Respiratory Syndrome Coronavirus and Group 2 Coronaviruses,"Although primary genomic analysis has revealed that severe acute respiratory syndrome coronavirus (SARS CoV) is a new type of coronavirus, the different protein trees published in previous reports have provided no conclusive evidence indicating the phylogenetic position of SARS CoV. To clarify the phylogenetic relationship between SARS CoV and other coronaviruses, we compiled a large data set composed of 7 concatenated protein sequences and performed comprehensive analyses, using the maximum-likelihood, Bayesian-inference, and maximum-parsimony methods. All resulting phylogenetic trees displayed an identical topology and supported the hypothesis that the relationship between SARS CoV and group 2 CoVs is monophyletic. Relationships among all major groups were well resolved and were supported by all statistical analyses.","{'query': 'coronavirus origin', 'narrative': ""seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans""}","{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/15116304/', 'pubmed_id': '15116304'}"
1,16,005b2j4b,0,how long does coronavirus remain stable on surfaces?,Monophyletic Relationship between Severe Acute Respiratory Syndrome Coronavirus and Group 2 Coronaviruses,"Although primary genomic analysis has revealed that severe acute respiratory syndrome coronavirus (SARS CoV) is a new type of coronavirus, the different protein trees published in previous reports have provided no conclusive evidence indicating the phylogenetic position of SARS CoV. To clarify the phylogenetic relationship between SARS CoV and other coronaviruses, we compiled a large data set composed of 7 concatenated protein sequences and performed comprehensive analyses, using the maximum-likelihood, Bayesian-inference, and maximum-parsimony methods. All resulting phylogenetic trees displayed an identical topology and supported the hypothesis that the relationship between SARS CoV and group 2 CoVs is monophyletic. Relationships among all major groups were well resolved and were supported by all statistical analyses.","{'query': 'how long does coronavirus survive on surfaces', 'narrative': 'Studies of time SARS-CoV-2 remains stable after being deposited from an infected person on everyday surfaces in a household or hospital setting, such as through coughing or touching objects.'}","{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/15116304/', 'pubmed_id': '15116304'}"
2,32,005b2j4b,0,"Does SARS-CoV-2 have any subtypes, and if so what are they?",Monophyletic Relationship between Severe Acute Respiratory Syndrome Coronavirus and Group 2 Coronaviruses,"Although primary genomic analysis has revealed that severe acute respiratory syndrome coronavirus (SARS CoV) is a new type of coronavirus, the different protein trees published in previous reports have provided no conclusive evidence indicating the phylogenetic position of SARS CoV. To clarify the phylogenetic relationship between SARS CoV and other coronaviruses, we compiled a large data set composed of 7 concatenated protein sequences and performed comprehensive analyses, using the maximum-likelihood, Bayesian-inference, and maximum-parsimony methods. All resulting phylogenetic trees displayed an identical topology and supported the hypothesis that the relationship between SARS CoV and group 2 CoVs is monophyletic. Relationships among all major groups were well resolved and were supported by all statistical analyses.","{'query': 'coronavirus subtypes', 'narrative': 'Papers that discuss subtypes of the virus, from named subtypes to speculative subtypes based on genomic or geographic clustering.'}","{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/15116304/', 'pubmed_id': '15116304'}"
3,

In [12]:
docs_with_questions = merged_df['corpus-id'].unique()

In [13]:
docs_with_questions.shape

(35480,)

## Explore all TREC COVID documents

In [19]:
all_docs_df = pd.read_csv(TREC_COVID_DOCUMENTS_FILE, sep='\t', header=None, names=['corpus-id', 'corpus-title-text'])

In [20]:
all_docs_df

,corpus-id,corpus-title-text
0,ug7v899j,"Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi ArabiaOBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were the most common symptoms, and crepitations (60%), and wheezes (40%) were the most common signs. Most patients with pneumonia had crepitations (79.2%) but only 25% had bronchial breathing. Immunocompromised patients were more likely than non-immunocompromised patients to present with pneumonia (8/9 versus 16/31, P = 0.05). Of the 24 patients with pneumonia, 14 (58.3%) had uneventful recovery, 4 (16.7%) recovered following some complications, 3 (12.5%) died because of M pneumoniae infection, and 3 (12.5%) died due to underlying comorbidities. The 3 patients who died of M pneumoniae pneumonia had other comorbidities. CONCLUSION: our results were similar to published data except for the finding that infections were more common in infants and preschool children and that the mortality rate of pneumonia in patients with comorbidities was high."
1,02tnwd4m,"Nitric oxide: a pro-inflammatory mediator in lung disease?Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lung."
2,ejv2xln0,"Surfactant protein-D and pulmonary host defenseSurfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can inf

## Sample documents to expand using LLM

In [22]:
EXPAND_ONLY_DOCUMENTS_WITH_QUERIES=False

In [25]:
if EXPAND_ONLY_DOCUMENTS_WITH_QUERIES:
    which_docs = rng.choice(docs_with_questions, 1000, replace=False)
    which_df = merged_df
    text_column = 'corpus-text'
else:
    which_docs = rng.choice(all_docs_df['corpus-id'].tolist(), 1000, replace=False)
    which_df = all_docs_df
    text_column = 'corpus-title-text'

In [26]:
which_docs

array(['pxniqk3i', 'twl9xy28', 'e7phn1y9', 'hlfk54ku', '6mrzkoy6',
       'vddwzeew', '1bat99us', '23eejhuw', 'mduqn7dq', 'vgutu093',
       'roh60mz8', 'b9rxrliw', 'l102f1e3', 'd2vkmx99', '3peprwq5',
       'ik9ja1g2', 'fqmmyjgj', 'pc4l0vf2', 'lvs3gy3m', 'n41yktbb',
       'j6gja3sg', 'cg05hoce', 'jro31ulk', '4xbqqaes', 'nh207jhs',
       'ffv4esde', 'ivor5s9v', '0ln9r2nq', 'yfxgm4rd', 'xvkjy1bh',
       '15eav27c', 'r0ej6jep', 's7j5cco5', '4id83ads', 'si4fymqs',
       'xi73bzud', 'hhs3owvr', 'l2dxthsv', 'h4t56r26', 'c5ubh6nr',
       'ivgaxqdw', 'pl7xpivx', 'ox3zqavc', '1xuz78mv', 'j8k4ym7u',
       'ta9khr6z', 'thbufxfk', '7h9fv2dr', 'oha0kfkz', 'fejho1of',
       'dvdis5hv', '649l3orp', 'eow3rdnj', 'uzknzybq', '7x8ro8p2',
       '98bhexbq', 'ku75k69n', 'obc9865a', '2j6qzzs3', 'wfm1m6xy',
       'nctsa6fz', 'hqfu6at0', 'bzax1u33', '4dznfmj4', 'pdnkvkiu',
       'wnh2cyy5', '1o9oqnd6', 'd97goifo', '4524vg4x', '3bpcf28i',
       '7unnfmks', 'l2pln8jd', '95sck64d', '1ovi549v', 'm7hcyy

In [27]:
which_df

,corpus-id,corpus-title-text
0,ug7v899j,"Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi ArabiaOBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were the most common symptoms, and crepitations (60%), and wheezes (40%) were the most common signs. Most patients with pneumonia had crepitations (79.2%) but only 25% had bronchial breathing. Immunocompromised patients were more likely than non-immunocompromised patients to present with pneumonia (8/9 versus 16/31, P = 0.05). Of the 24 patients with pneumonia, 14 (58.3%) had uneventful recovery, 4 (16.7%) recovered following some complications, 3 (12.5%) died because of M pneumoniae infection, and 3 (12.5%) died due to underlying comorbidities. The 3 patients who died of M pneumoniae pneumonia had other comorbidities. CONCLUSION: our results were similar to published data except for the finding that infections were more common in infants and preschool children and that the mortality rate of pneumonia in patients with comorbidities was high."
1,02tnwd4m,"Nitric oxide: a pro-inflammatory mediator in lung disease?Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lung."
2,ejv2xln0,"Surfactant protein-D and pulmonary host defenseSurfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can inf

## Generate some questions using OpenAI

In [28]:
OPENAI_API_QUERY_PARAMS={"model": None,
                         "prompt": None,
                         "temperature": 0.7,
                         "max_tokens": 512,
                         "top_p": 1.0,
                         "frequency_penalty": 0,
                         "presence_penalty": 0}

In [29]:
ZERO_SHOT_TEMPLATE="Considering the text in quotes below:\n\n\"{}\"\n\nWhich questions can you drawn about non-trivial conclusions from the text? Please write complete and independent questions, avoid implicit references to other questions or to the text."

In [30]:
OPENAI_RESPONSE_REGEX="[\n\r]*[0-9\.]+\s+(.+)[\n\r]?"

In [31]:
with open(API_KEYS_FILE) as inputFile:
    api_keys = json.load(inputFile)

In [32]:
openai.api_key = api_keys['OPENAI_API_KEY']

In [33]:
request_params = OPENAI_API_QUERY_PARAMS
request_params['model'] = 'text-davinci-003'

In [34]:
new_dataset = []

for document_id in which_docs:
    
    doc_text = which_df[which_df['corpus-id'] == document_id][text_column].to_numpy()[0]
    
    if doc_text is not np.nan:
        request_params['prompt'] = ZERO_SHOT_TEMPLATE.format(doc_text)

        print(request_params['prompt'])
        
        request_start_time = time.time()

        response = openai.Completion.create(**request_params)

        elapsed_time = time.time() - request_start_time
        
        print("-- Request time: {}".format(elapsed_time))
        
        print(response['choices'][0]['text'])
        
        m = re.finditer(OPENAI_RESPONSE_REGEX, response['choices'][0]['text'])
        
        questions = [match.group(1) for match in m]
        
        for question in questions:
            new_dataset.append({'query': question,
                                'positive_doc_id': document_id,
                                'negative_doc_ids': []})      

    else:
        print("Document {} has empty text...".format(document_id))

Considering the text in quotes below:

"Mass gatherings medicine: international cooperation and progress"

Which questions can you drawn about non-trivial conclusions from the text? Please write complete and independent questions, avoid implicit references to other questions or to the text.
-- Request time: 3.6963186264038086


1. What are the benefits of international cooperation in the field of medicine for mass gatherings? 
2. How has progress in mass gathering medicine been achieved through international cooperation? 
3. What challenges have been overcome with the help of international cooperation in the field of mass gatherings medicine?
4. What have been the most successful initiatives in international cooperation for mass gatherings medicine? 
5. How can international cooperation be further improved to promote progress in mass gatherings medicine?
Considering the text in quotes below:

"From the EditorCommentary: Addressing Inequities in the Era of COVID-19: The Pandemic and the

-- Request time: 3.4076714515686035


1. What proactive strategies and interventions can address the social determinants of health?
2. What differences exist in perceptions, experiences, and understandings of evidence-based practice among public health professionals?
3. How has the lack of a strong tradition of evidence-based practice in environmental health impacted the ability to secure funding?
4. What has caused tension between professionals with different backgrounds?
5. What investments are needed to develop evidence-based practice in environmental health?
Considering the text in quotes below:

"Management strategies of autoimmune bullous diseases during the outbreak of 2019 Novel Coronavirus disease (COVID-19)Since late December 2019, an outbreak of the novel coronavirus disease, which is mainly characterized by pulmonary lesions, has spread quickly worldwide. With the spread of the novel coronavirus, the outpatient lines of hospitals have mostly shut down, which means that rout

-- Request time: 3.5523312091827393


1. What is the difference in postoperative pain between Laparoendoscopic Single-Site Surgery (LESS) and Laparoscopic Cholecystectomy (LC)?
2. What is the difference in early inflammatory impact between Laparoendoscopic Single-Site Surgery and Laparoscopic Cholecystectomy?
3. Is there an association between body mass index and early inflammatory impact in the Laparoendoscopic Single-Site Surgery group?
Considering the text in quotes below:

"A scoping review on the prevalence of Shiga-toxigenic Escherichia coli in wild animal species.Zoonotic pathogens constitute the major source (60.3%) of emerging infectious diseases. Previous studies have investigated the prevalence of Shiga-toxigenic Escherichia coli (STEC) among wild animal species, but comprehensive data are needed to assess the role that these animals have in the transmission of STEC infections to the human population via faecal contamination of the environment, agri-food or water chain. Due 

-- Request time: 2.943394899368286


1. What compounds have been identified in Litsea verticillata Hance that can inhibit HIV-1 replication?
2. How do the two lignans identified in Litsea verticillata Hance demonstrate anti-EBV activity?
3. What species of Litsea were evaluated for their antiviral and cytotoxic activities?
4. What concentrations of the Litsea species extracts were evaluated for their antiviral and cytotoxic activities?
Considering the text in quotes below:

"Characteristics and Functions of Mucosa-Associated Lymphoid Tissue"

Which questions can you drawn about non-trivial conclusions from the text? Please write complete and independent questions, avoid implicit references to other questions or to the text.
-- Request time: 3.1195788383483887


1. What is the purpose of mucosa-associated lymphoid tissue?
2. What are the different types of mucosa-associated lymphoid tissue?
3. What are the primary functions of mucosa-associated lymphoid tissue?
4. How does mucosa-associ

-- Request time: 4.135212182998657


1. What is the difference in recurrence rate between 0.02% mitomycin C and limbal conjunctival autograft treatments for primary pterygium?
2. What are the complications associated with the use of 0.02% mitomycin C and limbal conjunctival autograft treatments for primary pterygium?
3. What factors should be taken into consideration when choosing between 0.02% mitomycin C and limbal conjunctival autograft treatments for primary pterygium?
Considering the text in quotes below:

"TeleStroke units serving as a model of care in rural areas: 10-year experience of the TeleMedical project for integrative stroke care.BACKGROUND AND PURPOSE Stroke Unit care improves stroke prognosis and is recommended for all patients with stroke. In rural areas, population-wide implementation of Stroke Units is challenging. Therefore, the TeleMedical Project for integrative Stroke Care (TEMPiS) was established in 2003 as a TeleStroke Unit network to overcome this barrier in S

...

-- Request time: 5.532174825668335


1. What components of synovial fluid facilitate low-friction movement between cartilaginous surfaces?
2. How has the use of synovial fluid in culturing chondrocytes differed in different species?
3. What challenges have been presented by the high viscosity of synovial fluid?
4. What advantages does culturing chondrocytes in synovial fluid offer over culturing them in artificial media?
5. How does combining chondrocyte culture with oxygen tension and mechanical loading create a realistic environment for cartilage growth?
6. What potential does cartilage tissue engineering offer for regenerating and restoring tissue function?
Considering the text in quotes below:

"Traffic-related air pollution and the coronavirus pandemia: shadows and lights"

Which questions can you drawn about non-trivial conclusions from the text? Please write complete and independent questions, avoid implicit references to other questions or to the text.
-- Request time: 5.359288

-- Request time: 4.833446502685547


1. What evidence was used to show a positive predictive path between digital social engagement, general smartphone use, and physical activity under lockdown conditions?
2. What is the relationship between social media use and total daily steps among individuals during lockdown?
3. How could monitoring of digital social engagement, general smartphone use, and physical activity help identify individuals at risk for negative effects of in-person social deprivation caused by the pandemic?
4. How could early intervention be implemented to protect vulnerable populations from the negative effects of pandemic-related stress?
Considering the text in quotes below:

"RNA interference for viral infections.The treatment of viral infections has relied on pre-emptive vaccination or use of a limited range of anti-viral drugs. However, the majority of viruses have no available drugs and treatment is merely supportive. RNA interference (RNAi) offers the ability to di

-- Request time: 4.258813142776489


1. What is the effect of IVIG treatment on total white blood cell counts and hemoglobin levels in KD patients?
2. How do the platelet, eosinophil, and monocyte levels differ between KD and EV patients before and after IVIG treatment?
3. What is the difference between the KD and EV patients in terms of the inflammatory response to IVIG treatment?
4. How does eosinophilia relate to IVIG therapy in KD and EV patients?
5. Are platelets involved in the inflammatory response to Kawasaki Disease?
Considering the text in quotes below:

"Effect of palatal implants on continuous positive airway pressure and compliance.OBJECTIVE Determine if the Pillar palatal implant system reduces continuous positive airway pressure (CPAP) pressure and improves patient compliance with CPAP therapy. STUDY DESIGN Randomized, double-blind, placebo-controlled study. SETTING Four geographically dispersed tertiary sleep disorder referral centers. METHODS Subjects with mild to mode

-- Request time: 3.618217945098877


1. What technical difficulties have been encountered when performing single port radical prostatectomy laparoendoscopic (LESS-RP)?
2. What advantages and disadvantages have been associated with LESS-RP?
3. How has the new robotic platform, the da Vinci SP, improved the application of LESS-RP?
4. What factors may have contributed to the low interest in the application of LESS-RP in prostate?
5. What opportunities does the evolution of robotic technologies for laparoscopy offer?
Considering the text in quotes below:

"Comparing Rapid Scoring Systems in Mortality Prediction of Critically Ill Patients With Novel Coronavirus DiseaseOBJECTIVES: Rapid and early severity‐of‐illness assessment appears to be important for critically ill patients with novel coronavirus disease (COVID‐19). This study aimed to evaluate the performance of the rapid scoring system on admission of these patients. METHODS: A total of 138 medical records of critically ill patients wi

-- Request time: 5.176900148391724


1. What is the frequency of anterior cerebral artery embolism caused by mechanical thrombectomy of middle cerebral artery occlusion?
2. What is the relevance of anterior cerebral artery embolism caused by mechanical thrombectomy of middle cerebral artery occlusion?
3. What are the associated devices that cause emboli in anterior cerebral artery branches?
4. How successful are attempts to recanalize the anterior cerebral artery?
5. What is the average volume of infarcts detected on follow-up imaging?
6. Does involvement of motor or supplementary motor areas occur in cases of anterior cerebral artery embolism?
7. Does successful endovascular recanalization of major anterior cerebral artery branches reduce the incidence of infarcts?
Considering the text in quotes below:

"Poliovirus Replication Requires the N-terminus but not the Catalytic Sec7 Domain of ArfGEF GBF1Viruses are intracellular parasites whose reproduction relies on factors provided by the

In [35]:
pd.DataFrame(new_dataset)

,query,positive_doc_id,negative_doc_ids
0,What are the benefits of international cooperation in the field of medicine for mass gatherings?,pxniqk3i,[]
1,How has progress in mass gathering medicine been achieved through international cooperation?,pxniqk3i,[]
2,What challenges have been overcome with the help of international cooperation in the field of mass gatherings medicine?,pxniqk3i,[]
3,What have been the most successful initiatives in international cooperation for mass gatherings medicine?,pxniqk3i,[]
4,How can international cooperation be further improved to promote progress in mass gatherings medicine?,pxniqk3i,[]
...,...,...,...
4909,What is the role of GBF1 in poliovirus replication?,lxb2otpr,[]
4910,How does the presence of BFA affect polio replication?,lxb2otpr,[]
4911,Does the N-terminus domain of GBF1 have any role in Arf GTPase activation?,lxb2otpr,[]
4912,How does the absence of p115 and Rab1b influence poliovirus replication?,lxb2otpr,[]


In [36]:
with open("eduseiti_1000_queries_expansion_20230502_01.jsonl", "w") as outputFile:
    for i, query in enumerate(new_dataset):
        if i > 0:
            outputFile.write("\n")
            
        json.dump(query, outputFile)